# Weapon Detection using YOLOv8

This notebook trains YOLOv8 models for weapon detection using the [Weapon Detection Dataset v2](https://universe.roboflow.com/joao-assalim-xmovq/weapon-2/dataset/2) from Roboflow.

## Setup
First, we'll install the required libraries and import dependencies.

In [ ]:
import os
import torch
import yaml
import time
import shutil
import random
from google.colab import files
import numpy as np

!pip install ultralytics roboflow

from ultralytics import YOLO
# Set up Roboflow for dataset download
from roboflow import Roboflow

## Dataset Functions

Functions to download and prepare the weapon detection dataset.

In [ ]:
def download_dataset(api_key, workspace="joao-assalim-xmovq", project="weapon-2", version=2):
    """Download and prepare dataset from Roboflow with proper folder structure."""
    print(f"Downloading dataset: {project} (v{version}) from {workspace}...")
    rf = Roboflow(api_key=api_key)
    proj = rf.workspace(workspace).project(project)
    dataset = proj.version(version).download("yolov8")
    dataset_path = dataset.location
    
    # Fix dataset directory structure if needed
    for split in ["train", "test"]:
        split_dir = os.path.join(dataset_path, split)
        imgs_dir = os.path.join(split_dir, "images")
        if os.path.exists(split_dir) and not os.path.exists(imgs_dir):
            os.makedirs(imgs_dir, exist_ok=True)
            for f in os.listdir(split_dir):
                if f.lower().endswith((".jpg",".jpeg",".png",".bmp")):
                    os.rename(os.path.join(split_dir, f), os.path.join(imgs_dir, f))
            print(f"Fixed {split}/images structure.")

    for split in ["train", "valid", "test"]:
        split_dir = os.path.join(dataset_path, split)
        if not os.path.isdir(split_dir): continue
        lbl_dir = os.path.join(split_dir, "labels")
        if not os.path.exists(lbl_dir):
            has_txt = any(f.endswith(".txt") for f in os.listdir(split_dir))
            if has_txt:
                os.makedirs(lbl_dir, exist_ok=True)
                for f in os.listdir(split_dir):
                    if f.endswith(".txt"):
                        os.rename(os.path.join(split_dir, f), os.path.join(lbl_dir, f))
                print(f"Fixed {split}/labels structure.")

    # Create validation split if missing
    train_img = os.path.join(dataset_path, "train", "images")
    train_lbl = os.path.join(dataset_path, "train", "labels")
    val_img   = os.path.join(dataset_path, "valid", "images")
    val_lbl   = os.path.join(dataset_path, "valid", "labels")
    
    os.makedirs(val_img, exist_ok=True)
    os.makedirs(val_lbl, exist_ok=True)
    
    if len(os.listdir(val_img)) == 0 and os.path.exists(train_img):
        imgs = [f for f in os.listdir(train_img) if f.lower().endswith((".jpg",".png",".bmp"))]
        if not imgs:
            print("No train images to split for validation.")
        else:
            cnt = max(1, int(len(imgs)*0.1))
            random.seed(42)
            chosen = random.sample(imgs, cnt)
            for im in chosen:
                shutil.copy2(os.path.join(train_img, im), os.path.join(val_img, im))
                lbl = os.path.splitext(im)[0] + ".txt"
                if os.path.exists(os.path.join(train_lbl, lbl)):
                    shutil.copy2(os.path.join(train_lbl, lbl), os.path.join(val_lbl, lbl))
            print(f"Created valid set with {cnt} images.")

    # Fix YAML paths
    yaml_path = os.path.join(dataset_path, "data.yaml")
    if os.path.exists(yaml_path):
        with open(yaml_path) as rf:
            data = yaml.safe_load(rf)
        modified = False
        for k in ["train","val","test"]:
            if k in data and data[k] and "/images" not in data[k]:
                data[k] = os.path.join(data[k], "images")
                modified = True
        if modified:
            with open(yaml_path, "w") as wf:
                yaml.dump(data, wf)
            print("Updated data.yaml image paths.")

    print(f"Dataset ready at: {dataset_path}")
    return dataset_path

def display_dataset_info(dataset_path):
    """Print class names and image counts per split."""
    yaml_path = os.path.join(dataset_path, "data.yaml")
    with open(yaml_path) as f:
        d = yaml.safe_load(f)
    print(f"- Classes ({len(d['names'])}): {d['names']}")
    for split in ["train","valid","test"]:
        p = os.path.join(dataset_path, split, "images")
        print(f"- {split} images: {len(os.listdir(p)) if os.path.isdir(p) else 0}")

## Training and Evaluation Functions

Functions to train the YOLOv8 model and evaluate its performance.

In [ ]:
def train_yolo(dataset_path, model_size='s', epochs=50, batch_size=16, img_size=640):
    """Train YOLOv8 model with T4-friendly defaults."""
    print("Checking GPU…")
    !nvidia-smi
    if model_size=='x': batch_size=min(batch_size,8)
    elif model_size=='l': batch_size=min(batch_size,12)
    elif model_size=='m': batch_size=min(batch_size,16)
    print(f"Batch size set to {batch_size} for yolov8{model_size}")
    model = YOLO(f"yolov8{model_size}.pt")
    data_yaml = os.path.join(dataset_path, "data.yaml")
    output_dir = "/content/yolov8_weapon_detection"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Training yolov8{model_size} for {epochs} epochs…")
    start = time.time()
    res = model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=output_dir,
        name=f"yolov8{model_size}_weapon_detection2",
        device=0 if torch.cuda.is_available() else "cpu",
        cache=True,
        amp=True,
        plots=True,
        verbose=True
    )
    print(f"Done in {(time.time()-start)/60:.2f} min")
    return model, output_dir

def evaluate_model(model, dataset_path):
    """Evaluate on valid set and print metrics."""
    data_yaml = os.path.join(dataset_path, "data.yaml")
    print("Running evaluation…")
    try:
        # Updated evaluation to handle API changes
        results = model.val(data=data_yaml, verbose=True)
        # Modern Ultralytics stores metrics differently
        if hasattr(results, 'box'):
            m = results.box
            print(f"P: {m.p:.4f}, R: {m.r:.4f}, mAP50: {m.map50:.4f}, mAP50-95: {m.map:.4f}")
        else:
            # Fallback for newer versions
            metrics = results.results_dict
            print(f"P: {metrics.get('precision', 0):.4f}, R: {metrics.get('recall', 0):.4f}")
            print(f"mAP50: {metrics.get('metrics/mAP50(B)', 0):.4f}, mAP50-95: {metrics.get('metrics/mAP50-95(B)', 0):.4f}")
        return results
    except AttributeError as e:
        print(f"Evaluation error with current metrics API: {e}")
        print("Using alternative metrics access...")
        # Try alternative approach for latest YOLO versions
        results = model.val(data=data_yaml)
        for metric_name, value in results.results_dict.items():
            if isinstance(value, (int, float)):
                print(f"{metric_name}: {value:.4f}")
        return results

def download_model(model_dir):
    """Download best.pt and last.pt via Colab."""
    for name in ("best.pt","last.pt"):
        p = os.path.join(model_dir, "weights", name)
        if os.path.exists(p):
            print(f"Downloading {name}…")
            files.download(p)
        else:
            print(f"{name} not found at {p}")

def export_model(model, fmt='pt'):
    """Export and download the model in the given format."""
    print(f"Exporting to {fmt}…")
    path = model.export(format=fmt, imgsz=640)
    print("Exported to", path)
    files.download(str(path))

## Main Function

The main function to run the entire workflow.

In [ ]:
def main():
    print("=== YOLO Weapon Detection Workflow ===")
    key = input("Roboflow API key: ")
    try:
        ds = download_dataset(key)
        display_dataset_info(ds)
        sz = input("Model size (n/s/m/l/x) [s]: ") or "s"
        ep = int(input("Epochs [50]: ") or 50)
        bs = int(input("Batch size [16]: ") or 16)
        img = int(input("Image size [640]: ") or 640)

        model, out = train_yolo(ds, sz, ep, bs, img)

        model_dir = os.path.join(out, f"yolov8{sz}_weapon_detection2")
        try:
            evaluate_model(model, ds)
        except Exception as e:
            print("⚠️  Eval error:", e)
            print("→ Skipping evaluation.")

        download_model(model_dir)

        if input("Export to pt? (y/n) [y]: ").lower() in ("","y"):
            export_model(model, "pt")

        print("✅ Workflow complete.")
    except Exception as e:
        print("‼️  ERROR:", e)
        import traceback; traceback.print_exc()

if __name__ == "__main__":
    main()